In [1]:
import numpy as np

In [2]:
from keras import backend as K
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

ModuleNotFoundError: No module named 'keras'

In [3]:
from keras.applications import VGG16
from scipy.optimize import fmin_l_bfgs_b

ModuleNotFoundError: No module named 'keras'

In [3]:
# paths
c_im_path = 'data/megan_fox.jpg'
s_im_path = 'data/melodyonight.jpg'


In [4]:
# target dims
targetHeight = 512
targetWidth = 512
targetSize = (targetHeight, targetWidth)

In [5]:
def load_preprocess(img_path,
                    targetSize=targetSize):
    '''
    Loads image as PIL.image, converts into np.ndarray
        then preprocesses it as a Vgg16 input shape
        after expanding row dimension by1
    '''
    im = load_img(path=img_path,
                     target_size=targetSize)
    im_arr = img_to_array(im)
    im_arr = K.variable(preprocess_input(
                            np.expand_dims(im_arr, axis=0)),
                            dtype='float32')
    return im_arr

In [21]:
cImArr = load_preprocess(c_im_path) # content image
sImArr = load_preprocess(s_im_path) # style image

In [40]:
gIm0 = np.random.randint(256,
                size=(targetWidth, targetHeight, 3)).astype('float64')
gIm0 = preprocess_input(np.expand_dims(g_im_0, axis=0))
gImPlaceholder = K.placeholder(
        shape=(1, targetWidth, targetHeight, 3))

In [35]:
def get_feature_reps(x, layer_names, model):
    '''
    Get feature representations of
    input x for one or more layers in a given model.
    '''
    featMatrices = []
    for ln in layer_names: # iterate through layers
        selectedLayer = model.get_layer(ln)
        featRaw = selectedLayer.output
        featRawShape = K.shape(featRaw).eval(session=tf_session)
        
        # rows
        N_1 = featRawShape[-1] 
        # columns
        M_1 = featRawShape[1]*featRawShape[2] # height * width
        
        featMatrix = K.reshape(featRaw, (M_1, N_1))
        featMatrix = K.transpose(featMatrix)
        
        featMatrices.append(featMatrix)
        
    return featMatrices      

In [9]:
def get_content_loss(F, P):
    cLoss = 0.5*K.sum(K.square(F - P))
    return cLoss

In [77]:
def get_Gram_matrix(F):
    G = K.dot(F, K.transpose(F))
    return G

In [89]:
def get_style_loss(ws, Gs, As):
    sLoss = K.variable(0.)
    for w, G, A in zip(ws, Gs, As):
        M_1 = K.int_shape(G)[1]
        N_1 = K.int_shape(G)[0]
        
        G_gram = get_Gram_matrix(G)
        A_gram = get_Gram_matrix(A)
        
        norm_fact = N_1**2 * M_1**2
        weight_sum = w*0.25*K.sum(K.square(G_gram - A_gram))
        sLoss+= weight_sum / norm_fact
    
    return sLoss

In [79]:
def get_total_loss(gImPlaceholder, alpha=1.0, beta=1000.0):
    """
    Total Style Transfer Loss function.
    alpha: Adjusts weight of the content Loss
    beta: Adjusts weight of the style Loss
    """
    F = get_feature_reps(gImPlaceholder,
                         layer_names=[cLayerName],
                        model = gModel)[0]
    Gs = get_feature_reps(gImPlaceholder,
                         layer_names=sLayerNames,
                         model=gModel)
    # content loss
    contentLoss = get_content_loss(F, P)
    
    # style loss
    styleLoss = get_style_loss(ws, Gs, As)
    
    # weighted loss
    totalLoss = alpha*contentLoss + beta*styleLoss
    
    return totalLoss

In [80]:
def calculate_loss(gImArr):
    """
    Calculate total loss
    """
    if gImArr.shape != (1, targetWidth, targetHeight, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetWidth, 3))
    loss_fcn = K.function([gModel.input], 
                          [get_total_loss(gModel.input)])
    return loss_fcn([gImArr])[0].astype('float64')

In [81]:
def get_grad(gImArr):
    """
    Calculate the gradient of the loss function
    with respect to the generated image
    """
    if gImArr.shape != (1, targetWidth, targetHeight, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    grad_fcn = K.function([gModel.input],
                         K.gradients(
                             get_total_loss(
                                 gModel.input), [gModel.input]))
    
    grad = grad_fcn([gImArr])[0].flatten().astype('float64')
    return grad

In [82]:
# tensor session
tf_session = K.get_session()

In [83]:
cModel = VGG16(include_top=False,
               weights='imagenet',
              input_tensor=cImArr)
sModel = VGG16(include_top=False,
                weights='imagenet',
              input_tensor=sImArr)

In [84]:
gModel = VGG16(include_top=False,
                weights='imagenet',
              input_tensor=gImPlaceholder)

In [85]:
cLayerName = 'block4_conv2'

In [86]:
sLayerNames = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1'
]

In [87]:
P = get_feature_reps(x=cImArr,
                     layer_names=[cLayerName],
                    model=cModel)[0]
As = get_feature_reps(x=sImArr,
                     layer_names=sLayerNames,
                     model=sModel)
ws = np.ones(len(sLayerNames))/float(len(sLayerNames))

In [ ]:
iterations = 600
x_val = gIm0.flatten()
xopt, f_val, info = fmin_l_bfgs_b(calculate_loss,
                                  x_val,
                                  fprime=get_grad,
                                  maxiter=iterations,
                                  disp=True)